### Задача 6.1

Рассмотрим задачу предсказание числа заболевших некоторой болезнью от некоторых экологических анализов (см. файл). Гарантируется, что предсказание описывается линейной моделью.

Так как проведение анализов не является бесплатным, то стоит вопрос о том какие из анализов являются лишними (на уровне значимости $\alpha=0.05$) для предсказания линейной модели.

Требуется:

* Записать задачу формально;
* Провести отбор признаков линейной модели.

Все выкладки должны быть сделаны аналитически, без использования компьютера. (допускается использование компютера для подстановвки численых значений в финальную формулу)

In [8]:
from statsmodels.api import GLM
from scipy.stats.stats import pearsonr
from numpy.linalg import inv, norm
import scipy.stats as st
import numpy as np
from numpy.linalg import norm, inv

In [9]:
import pandas as pd
sick = pd.read_csv('data/sick.csv')
sick.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,Y
0,-0.5,-0.1,-1.2,-0.6,-1.1,1.4,-1.4,1.2,-0.2,-0.2,0.0
1,1.0,0.4,0.5,-1.1,0.6,-0.1,-0.2,-0.7,-0.5,0.4,1.0
2,0.3,-0.9,0.8,-0.3,-0.2,-1.4,0.4,1.6,1.0,-0.3,3.0
3,-1.1,-0.5,0.5,1.8,0.3,-0.3,-0.1,0.4,1.0,0.3,3.0
4,1.9,0.6,0.4,0.7,-2.9,0.5,-0.9,-1.5,0.9,-3.1,1.0


Сначала просто прикинем результат с помощью копмьютера)

In [10]:
formula = 'Y ~ ' + ''.join([f'x{x} + ' for x in range(1, 11)]) + '1'

model = GLM.from_formula(formula, data=sick).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      Y   No. Observations:                   30
Model:                            GLM   Df Residuals:                       19
Model Family:                Gaussian   Df Model:                           10
Link Function:               identity   Scale:                          4.2355
Method:                          IRLS   Log-Likelihood:                -57.369
Date:                Mon, 04 Apr 2022   Deviance:                       80.474
Time:                        23:45:56   Pearson chi2:                     80.5
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.9411      0.491      3.954      0.000       0.979       2.903
x1            -0.8668      0.380     -2.284      0.022      -1.611      -0.123
x2             0.3373      0.456      0.739      0.460      -0.557       1.232
x3             2.9763      0.527      5.651      0.000       1.944       4.009
x4             0.4234      0.355      1.193      0.233      -0.272       1.119
x5             0.0855      0.516      0.166      0.869      -0.927       1.098
x6            -0.4043      0.430     -0.940      0.347      -1.247       0.438
x7            -0.3956      0.438     -0.903      0.367      -1.255       0.464
x8             0.1615      0.472      0.342      0.732      -0.763       1.086
x9             0.0899      0.461      0.195      0.845      -0.814       0.994
x10           -0.0148      0.550     -0.027      0.979      -1.092       1.063
==============================================================================
"""

Отсюда видно, что значимыми являются x1 и x3

In [11]:
model = GLM.from_formula('Y ~ x1 + x3 + 1', data=sick).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      Y   No. Observations:                   30
Model:                            GLM   Df Residuals:                       27
Model Family:                Gaussian   Df Model:                            2
Link Function:               identity   Scale:                          3.5502
Method:                          IRLS   Log-Likelihood:                -59.993
Date:                Mon, 04 Apr 2022   Deviance:                       95.856
Time:                        23:45:57   Pearson chi2:                     95.9
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.7281      0.367      4.711      0.000       1.009       2.447
x1            -0.7420      0.297     -2.497      0.013      -1.324      -0.160
x3             3.1980      0.433      7.392      0.000       2.350       4.046
==============================================================================
"""

Будем добавлять в модель по одну признаку из массива признаков, отсортированному по убыванию модуля коэффициента корреляции Пирсона с целлевой переменной (то есть сначала добаляем в модель признак, наиболее скоррелированный с целевой переменной)

Затем с помощью критерия Фишера проверяем гипотезу о том, что все остальные все остальные коэффициенты в модели равны нулю (на уровне значимости $\alpha = 0.05$)

Если гипотеза отклоняется, то добавляем в модель следующий признак. И так до тех пор, пока мы не примем гипотезу на заданном уровне значимости. 

In [12]:
correlations = [st.pearsonr(sick[f'x{i}'], sick['Y'])[0] for i in range(1, sick.shape[1])]
max_corr = max(correlations)
print('max_corr = {:.3f}, index = {}'.format(max_corr, correlations.index(max_corr)+1))

max_corr = 0.786, index = 3


Значит сначала строим модель с одним признаком (и ещё intercept). То есть $y = \beta_0 + \beta_3 \cdot x_3$

Критерий Фишера:

$H_0: \beta_2 = 0$

$H_1:$ нулевая гипотеза неверна

$\underset{(k+1)\times1}{\beta^T} = \Bigg(\underset{(k+1-k_1) \times 1}{\beta_1^T}, \underset{k_1 \times 1}{\beta_2^T}\Bigg)^T $,
$\underset{n\times(k+1)}{X} = \Bigg(\underset{n \times (k+1-k_1)}{X_1}, \underset{n \times k_1}{X_2}\Bigg)^T $,
$RSS_r = ||y-X_1 \beta_1||_2^2$,  $RSS_{ur} = ||y-X \beta||_2^2$

Статистика: $$F = \frac{(RSS_r - RSS_{ur})/k_1}{RSS_{ur}/(n-k-1)}$$

Нулевое распределение: $F(k_1, n-k-1)$ - распределение Фишера

In [13]:
def Fisher_criterion(columns='x3'):
    X1 = np.c_[np.ones(sick.shape[0]), sick[columns].to_numpy()] 
    X = np.c_[np.ones(sick.shape[0]), sick.drop(columns='Y').to_numpy()] 

    b1 = inv(X1.T @ X1) @ X1.T @ sick['Y']
    b = inv(X.T @ X) @ X.T @ sick['Y']

    RSS_r = norm(sick['Y'] - X1 @ inv(X1.T @ X1) @ X1.T @ sick['Y'])
    RSS_ur = norm(sick['Y'] - X @ inv(X.T @ X) @ X.T @ sick['Y'])
    
    dfn = X.shape[1] - X1.shape[1]
    dfd = X.shape[0] - X.shape[1] - 1 
    
    F = ((RSS_r - RSS_ur)/dfn)/(RSS_ur/dfd)
    F_cr = st.f.ppf(0.95, dfn, dfd)    
    p_value = st.f.sf(F, dfn, dfd)
    
    return F, F_cr, p_value

In [22]:
print('T = {}, T_cr = {}, p-value = {}'.format(*Fisher_criterion('x3')))

T = 0.4218351979686487, T_cr = 2.4562811491592678, p-value = 0.9063962989640306


Хм...Ну вот мы и получили, что в принципе все остальные коэффициенты можно занулить. 

Хотя, очевидно, можно построить модель и лучше, добавив в неё переменныу x1

In [23]:
print('T = {}, T_cr = {}, p-value = {}'.format(*Fisher_criterion(['x3', 'x1']) ))

T = 0.20564382224294597, T_cr = 2.5101578953835753, p-value = 0.9859340472077198


Таким образом, признаки ['x3', 'x1'] - наиболее значимые 

Коэффициенты у этой модели следующие (первых из них - intercept):

In [25]:
X1 = np.c_[np.ones(sick.shape[0]), sick[['x3', 'x1']].to_numpy()] 
b1 = inv(X1.T @ X1) @ X1.T @ sick['Y']
print(b1)

[ 1.72806451  3.19797545 -0.74204352]
